# Neural network prediction for the sPOD-DL-ROM for WIldfire

In [ ]:
import sys
sys.path.append('../DL/')
sys.path.append('../sPOD/lib/')
sys.path.append('../DL-ROM/LIB/')

In [ ]:
from wildfire2D import wildfire2D
import numpy as np

## Data generation / Shifted POD of the data

In [ ]:
import os

q = np.load(os.path.abspath(".") + '/wildfire_data/2D/' + 'SnapShotMatrix558_49.npy')
shifts_test= np.load(os.path.abspath(".") + '/wildfire_data/2D/' + 'Shifts558_49.npy')

df = wildfire2D(q, shifts_test, param_test_val=558.49, var=0)

In [ ]:
############################# Run shifted POD on the data ########################## (only once)
impath = "./wildfire_data/2D/save_Wildfire/T/"
import os
import pickle
os.makedirs(impath, exist_ok=True)

U_list, TA_list_training, TA_list_interp, spod_modes = df.run_sPOD(spod_iter=2000)

with open(impath + 'U_list.data', 'wb') as filehandle:
    pickle.dump(U_list, filehandle)
with open(impath + 'TA_list_training.data', 'wb') as filehandle:
    pickle.dump(TA_list_training, filehandle)
with open(impath + 'TA_list_interp.data', 'wb') as filehandle:
    pickle.dump(TA_list_interp, filehandle)
with open(impath + 'spod_modes.data', 'wb') as filehandle:
    pickle.dump(spod_modes, filehandle)
with open(impath + 'Q_polar_train.data', 'wb') as filehandle:
    pickle.dump(df.q_polar_train, filehandle)

# Load the data for NN predictions and analysis

## Load the training data

In [ ]:
impath = "./wildfire_data/2D/save_Wildfire/T/"
import os
import pickle

with open(impath + 'U_list.data', 'rb') as filehandle:
    U_list = pickle.load(filehandle) 
with open(impath + 'TA_list_training.data', 'rb') as filehandle:
    TA_list_training = pickle.load(filehandle)  
with open(impath + 'TA_list_interp.data', 'rb') as filehandle:
    TA_list_interp = pickle.load(filehandle) 
with open(impath + 'spod_modes.data', 'rb') as filehandle:
    spod_modes = pickle.load(filehandle) 
with open(impath + 'Q_polar_train.data', 'rb') as filehandle:
    Q_polar_train = pickle.load(filehandle) 

In [ ]:
# Plot the offline variables
df.plot_offline_data(TA_list_training)

In [ ]:
from sklearn.utils.extmath import randomized_svd

TA_TRAIN = np.concatenate(TA_list_training, axis=0)
SHIFTS_TRAIN = df.shifts_train[0][0]
PARAMS_TRAIN = df.params_train

u, s, vt = randomized_svd(Q_polar_train, n_components=sum(spod_modes) + 1, random_state=None)
U_POD_TRAIN = u
TA_POD_TRAIN = np.diag(s) @ vt

In [ ]:
######################################### Temporary cell for dof study ############################################
frame_wise_sPOD = [19, 5]
Nmf = spod_modes
time_amplitudes_1 = TA_TRAIN[:Nmf[0], :]
time_amplitudes_2 = TA_TRAIN[Nmf[0]:, :]
frame_amplitudes_list_training = [
    time_amplitudes_1[:frame_wise_sPOD[0], :],
    time_amplitudes_2[:frame_wise_sPOD[1], :]
]

TA_TRAIN = np.concatenate(frame_amplitudes_list_training, axis=0)
U_list = [
    U_list[0][:, :frame_wise_sPOD[0]], 
    U_list[1][:, :frame_wise_sPOD[1]]
]
spod_modes = frame_wise_sPOD

frame_amplitudes_list_interp = []
for frame in range(2):
    Nmodes = spod_modes[frame]
    VT = frame_amplitudes_list_training[frame]
    amplitudes = [np.reshape(VT[n, :], [df.Nsamples_train, len(df.t)]).T for n in range(Nmodes)]
    frame_amplitudes_list_interp.append(amplitudes)

TA_list_interp = frame_amplitudes_list_interp
U_POD_TRAIN = U_POD_TRAIN[:, :sum(spod_modes) + 1]
TA_POD_TRAIN = TA_POD_TRAIN[:sum(spod_modes) + 1, :]

In [ ]:
from Helper import *
############################################
X_new = df.X - df.x_c  # Shift the origin to the center of the image
Y_new = df.Y - df.y_c
r = np.sqrt(X_new ** 2 + Y_new ** 2).flatten()  # polar coordinate r
theta = np.arctan2(Y_new, X_new).flatten()  # polar coordinate theta
r_i = np.linspace(np.min(r), np.max(r), df.Nx)
theta_i = np.linspace(np.min(theta), np.max(theta), df.Ny)
dr = r_i[1] - r_i[0]
dtheta = theta_i[1] - theta_i[0]
d_del = np.asarray([dr, dtheta])
L = np.asarray([r_i[-1], theta_i[-1]])
data_shape = [df.Nx, df.Ny, 1, df.Nsamples_train*df.Nt]
Ndims = 2

q1 = U_list[0] @ frame_amplitudes_list_training[0] 
q2 = U_list[1] @ frame_amplitudes_list_training[1]
q_train = [np.reshape(q1, newshape=data_shape), np.reshape(q2, newshape=data_shape)]

trafos = [
transforms(data_shape, L, shifts=df.shifts_train[0], dx=d_del,
                          use_scipy_transform=True),
transforms(data_shape, L, shifts=df.shifts_train[1], trafo_type="identity",
                          dx=d_del,
                          use_scipy_transform=True)
]

NumFrames = 2
q_sPOD = 0
for frame in range(NumFrames):
    q_sPOD += trafos[frame].apply(q_train[frame])
############################################
q_POD = U_POD_TRAIN @ TA_POD_TRAIN
q_POD = np.reshape(q_POD, newshape=data_shape)
############################################
q_original = np.reshape(Q_polar_train, newshape=data_shape)
############################################

res = q_original - q_sPOD
err_sPOD = np.linalg.norm(np.reshape(res, -1)) / np.linalg.norm(np.reshape(q_original, -1))

res = q_original - q_POD
err_POD = np.linalg.norm(np.reshape(res, -1)) / np.linalg.norm(np.reshape(q_original, -1))

print("Relative reconstruction error indicator for full snapshot(sPOD) is {}".format(err_sPOD))
print("Relative reconstruction error indicator for full snapshot(POD) is {}".format(err_POD))

## Load the testing data

In [ ]:
############################# Run shifted POD on the test data ########################## (only once)
import os
impath = "./wildfire_data/2D/save_Wildfire/T/"
import pickle
os.makedirs(impath, exist_ok=True)

Q_frames_test_polar, Q_frames_test_cart = df.test_data(spod_iter=1000)

with open(impath + 'Q_frames_test_polar.data', 'wb') as filehandle:
    pickle.dump(Q_frames_test_polar, filehandle)
with open(impath + 'Q_frames_test_cart.data', 'wb') as filehandle:
    pickle.dump(Q_frames_test_cart, filehandle)
with open(impath + 'Q_test_polar.data', 'wb') as filehandle:
    pickle.dump(df.q_polar_test, filehandle)

In [ ]:
impath = "./wildfire_data/2D/save_Wildfire/T/"
import os
import pickle

with open(impath + 'Q_frames_test_polar.data', 'rb') as filehandle:
    Q_frames_test_polar = pickle.load(filehandle) 
with open(impath + 'Q_frames_test_cart.data', 'rb') as filehandle:
    Q_frames_test_cart = pickle.load(filehandle) 
with open(impath + 'Q_test_polar.data', 'rb') as filehandle:
    Q_test_polar = pickle.load(filehandle) 

In [ ]:
# Plot the frames for test parameter
df.plot_sPOD_frames(Q_frames_test_cart, plot_every=10, var_name="T")

In [ ]:
mu_vecs_test = np.asarray([df.param_test_val])
params_test = [np.squeeze(np.asarray([[np.ones_like(df.t) * mu], [df.t]])) for mu in mu_vecs_test]
PARAMS_TEST = np.concatenate(params_test, axis=1)

q1_test = Q_frames_test_polar[0]
q2_test = Q_frames_test_polar[1]
time_amplitudes_1_test = U_list[0].transpose() @ q1_test
time_amplitudes_2_test = U_list[1].transpose() @ q2_test

TA_TEST = np.concatenate((time_amplitudes_1_test, time_amplitudes_2_test), axis=0)
SHIFTS_TEST = df.shifts_test[0][0]
TA_POD_TEST = U_POD_TRAIN.transpose() @ Q_test_polar

# -----------------------------------------------------------

In [ ]:
print("Grid, Nx : {}, Ny : {}, Nt : {}".format(len(df.x), len(df.y), len(df.t)))
print("Number of sPOD frames : {}".format(len(spod_modes)))
print("Number of modes (frame wise) : {}, {}".format(spod_modes[0], spod_modes[1]))
print("Size of training matrix : {} x {}".format(int(TA_TRAIN.shape[0]), int(TA_TRAIN.shape[1])))

In [ ]:
shifts_train = np.reshape(SHIFTS_TRAIN, newshape=[1, -1])
shifts_test = np.reshape(SHIFTS_TEST, newshape=[1, -1])

ta_train = np.concatenate((TA_TRAIN, shifts_train), axis=0)
ta_test = np.concatenate((TA_TEST, shifts_test), axis=0)

# Network call

# Feed forward neural network

In [ ]:
# params_sPOD = {
#         'scaling': True,  # true if the data should be scaled
#         'full_order_model_dimension': len(df.x) * len(df.y),  # N_h
#         'reduced_order_model_dimension': ta_train.shape[0],  # N
#         'totalModes': ta_train.shape[0] - len(spod_modes) + 1,  # Total number of modes for all the frames
#         'num_early_stop': 3000  # Early stop criteria 
#     }
# params_POD = {
#         'scaling': True,  # true if the data should be scaled
#         'full_order_model_dimension': len(df.x) * len(df.y),  # N_h
#         'reduced_order_model_dimension': TA_POD_TRAIN.shape[0],  # N
#         'totalModes': TA_POD_TRAIN.shape[0],  # Total number of modes for all the frames
#         'num_early_stop': 3000  # Early stop criteria 
#     }

In [ ]:
# # training the model
# from network import run_model 
# print("#################################")
# print("sPOD-DL-ROM")
# model_sPOD, scaling_sPOD = run_model(ta_train, PARAMS_TRAIN, epochs=100000, lr=0.05, loss_type='L1', 
#                        logs_folder='./DNN_result/wildfire2D/training_results_sPOD/T',
#                       pretrained_load=False, pretrained_weights=None, params=params_sPOD, batch_size=100)
# print("#################################\n")
# print("#################################")
# print("POD-DL-ROM")
# model_POD, scaling_POD = run_model(TA_POD_TRAIN, PARAMS_TRAIN, epochs=100000, lr=0.05, loss_type='L1', 
#                       logs_folder='./DNN_result/wildfire2D/training_results_POD/T',
#                      pretrained_load=False, pretrained_weights=None, params=params_POD, batch_size=100)
# print("#################################\n")

In [ ]:
# # loading the model
# import torch
# import pathlib
# import os

# log_folder_base_sPOD = 'DNN_result/wildfire2D/training_results_sPOD/T/'
# log_folder_trained_model_sPOD = sorted(pathlib.Path(log_folder_base_sPOD).glob('*/'), key=os.path.getmtime)[-1]
# PATH_sPOD = str(log_folder_trained_model_sPOD) + '/trained_weights/' + 'weights.pt'


# log_folder_base_POD = 'DNN_result/wildfire2D/training_results_POD/T/'
# log_folder_trained_model_POD = sorted(pathlib.Path(log_folder_base_POD).glob('*/'), key=os.path.getmtime)[-1]
# PATH_POD = str(log_folder_trained_model_POD) + '/trained_weights/' + 'weights.pt'

# print(PATH_sPOD)
# print(PATH_POD)

In [ ]:
# from network import scale_params

# if '/trained_weights/weights.pt' in PATH_sPOD: address_sPOD = PATH_sPOD.replace('/trained_weights/weights.pt', '')
# scaling_sPOD = np.load(address_sPOD + '/variables/' + 'scaling.npy', allow_pickle=True)

# if '/trained_weights/weights.pt' in PATH_POD: address_POD = PATH_POD.replace('/trained_weights/weights.pt', '')
# scaling_POD = np.load(address_POD + '/variables/' + 'scaling.npy', allow_pickle=True)

# PARAMS_TEST_sPOD = scale_params(PARAMS_TEST, params_sPOD, scaling_sPOD)
# PARAMS_TEST_POD = scale_params(PARAMS_TEST, params_POD, scaling_POD)

In [ ]:
# # testing the model
# from network import test_model 
# import time 

# tic = time.process_time()
# rel_err_sPOD, results_predicted_sPOD = test_model(ta_test, PARAMS_TEST_sPOD, 
#                                                   trained_model=None, saved_model=True,
#                                                  PATH_TO_WEIGHTS=PATH_sPOD, params=params_sPOD, 
#                                                   scaling=scaling_sPOD, batch_size=500) 
# toc = time.process_time()
# print(f"Time consumption in testing sPOD DL model : {toc - tic:0.4f} seconds")

# tic = time.process_time()
# rel_err_POD, results_predicted_POD = test_model(TA_POD_TEST, PARAMS_TEST_POD, 
#                                                 trained_model=None, saved_model=True,
#                                                PATH_TO_WEIGHTS=PATH_POD, params=params_POD,
#                                                scaling=scaling_POD, batch_size=500)
# toc = time.process_time()
# print(f"Time consumption in testing POD DL model : {toc - tic:0.4f} seconds")


# print(rel_err_sPOD, rel_err_POD)

# -----------------------------------------------------------

In [ ]:
# # This cell is reserved for data manipulations for the online analysis
# frame_amplitudes_predicted_sPOD = results_predicted_sPOD[:-1, :]
# shifts_predicted_sPOD = results_predicted_sPOD[-1:, :]
# frame_amplitudes_predicted_POD = results_predicted_POD

In [ ]:
# Q_recon_sPOD_cart, Q_recon_POD_cart, Q_recon_interp_cart, errors = df.plot_online_data(frame_amplitudes_predicted_sPOD, 
#                                         frame_amplitudes_predicted_POD,
#                                         TA_TEST,
#                                         TA_POD_TEST,
#                                         TA_list_interp,                          
#                                         shifts_predicted_sPOD,
#                                         SHIFTS_TEST,
#                                         spod_modes,
#                                         U_list,
#                                         U_POD_TRAIN,
#                                         Q_test_polar,
#                                         Q_frames_test_polar)

In [ ]:
# df.plot_recon(Q_recon_sPOD_cart, Q_recon_POD_cart, Q_recon_interp_cart, t_a=10, t_b=100)

# Erros Plots

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import os
# from Helper import save_fig
# from statistics import mean

# impath = "../plots/images_wildfire2D/"
# os.makedirs(impath, exist_ok=True) 

# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.serif": ["Computer Modern"]})

# SMALL_SIZE = 16   # 16
# MEDIUM_SIZE = 18   # 18
# BIGGER_SIZE = 20   # 20

# plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
# plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
# plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
# plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
# plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
# plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
# plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
# truncated_modes = np.array([3, 6, 14, 20, 30, 40, 46])
# E_sPOD_NN = np.array([0.99486, 0.96774, 0.18548, 0.18436, 0.06719, 0.0554, 0.03354])
# E_POD_NN = np.array([0.55695, 0.38588, 0.221744, 0.167523, 0.09577, 0.10325, 0.08427])
# E_sPOD_LI = np.array([0.27787, 0.2768, 0.05805, 0.05771, 0.05576, 0.05573, 0.05573])

# err = errors[0]
# err_min = [max(x) for x in err]
# err_max = [min(x) for x in err]
# err_mean = [mean(x) for x in err]

# fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# axs[0].semilogy(truncated_modes, E_sPOD_NN, color="green", linestyle='--', marker="*", label=r"$E^{\mathrm{sPOD-NN}}_{\mathrm{tot}}$")
# axs[0].semilogy(truncated_modes, E_POD_NN, color="blue", linestyle='--', marker="*", label=r"$E^{\mathrm{POD-NN}}_{\mathrm{tot}}$")
# axs[0].semilogy(truncated_modes, E_sPOD_LI, color="yellow", linestyle='--', marker="*", label=r"$E^{\mathrm{sPOD-LI}}_{\mathrm{tot}}$")
# axs[0].set_xlabel('Number of modes')
# axs[0].set_ylabel('Errors')
# axs[0].grid()
# axs[0].legend(loc='upper right')

# axs[1].semilogy(df.t, err_min, color="green", linestyle='--', label=r"$min(E_{\mathrm{t}})$")
# axs[1].semilogy(df.t, err_max, color="black", linestyle='--', label=r"$max(E_{\mathrm{t}})$")
# axs[1].semilogy(df.t, err_mean, color="red", linestyle='--', label=r"$mean(E_{\mathrm{t}})$")
# axs[1].set_xlabel(r"time $t$")
# axs[1].grid()
# axs[1].legend(loc='lower right')


# save_fig(filepath=impath + 'Rel_err', figure=fig)
# fig.savefig(impath + "Rel_err" + ".eps", format='eps',dpi=600, transparent=True)